In [1]:
from annoy import AnnoyIndex
tfidf = AnnoyIndex(7000, 'angular')
print("load model...")
tfidf.load('../../pickles/swarog_data/swarog_tfidf.ann')

from annoy import AnnoyIndex
bert = AnnoyIndex(768, 'angular')
print("load model...")
bert.load('../../pickles/swarog_data/swarog_bertcls.ann')


load model...
load model...


True

In [2]:
import pickle5 as pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import sqlite3

conn = sqlite3.connect('../../pickles/swarog_data/swarog.sqlite')
rawdata = pd.read_sql_query("select dataset, rowid as gid, body from raw", conn)
conn.close()

ModuleNotFoundError: No module named 'pickle5'

In [19]:
rawdata.head()

,dataset,gid,body
0,covid_fake_news,1,A post claims compulsory vacination violates t...
1,covid_fake_news,2,A photo claims that this person is a doctor wh...
2,covid_fake_news,3,Post about a video claims that it is a protest...
3,covid_fake_news,4,All deaths by respiratory failure and pneumoni...
4,covid_fake_news,5,The dean of the College of Biologists of Euska...


In [11]:

with open(f'../../pickles/swarog_data/swarog_tfidf.pickle', 'rb') as handle:
        tfidfvec = pickle.load(handle)
    
with open(f'../../pickles/swarog_data/swarog_bertcls.pickle', 'rb') as handle:
        bertclsvec = pickle.load(handle)


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
import pickle5 as pickle

# Interface lemma tokenizer from nltk with sklearn
class LemmaTokenizer:
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`']
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if t not in self.ignore_tokens]
    
with open(f'../../pickles/swarog_data/tfidf_vectorizer.pickle', 'rb') as handle:
        tfidf_vectorizer = pickle.load(handle)

In [ ]:
class MyCustomUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == "__main__":
            module = "lematizer"
        return super().find_class(module, name)

with open('out.pkl', 'rb') as f:
    unpickler = MyCustomUnpickler(f)
    obj = unpickler.load()

print(obj)
print(obj.name)


In [4]:
tfidf_vectorizer

TfidfVectorizer(max_features=7000,
                stop_words=['s', 'above', 'it', "'s", 'you', 'they', 't', 'up',
                            'were', 'what', 'of', 'himself', 'she', "'s",
                            'some', 'had', 'those', 'from', 'how', 'shouldn',
                            'our', 'below', 'over', 'with', 'have', "n't",
                            'whom', 'you', "'re", 'under', ...],
                tokenizer=<__main__.LemmaTokenizer object at 0x000001C846CF7BE0>)

In [3]:
LemmaTokenizer()

In [11]:
tfidf_vectorizer.transform(["Obama is playing tenis"]).toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [11]:
tfidfvec.shape, bertclsvec.shape

((185460, 7000), (186477, 768))

In [ ]:
bertclsvec[1]

In [14]:
tfidf.get_nns_by_item(2, 10, search_k=-1, include_distances=True)

([2, 3560, 5213, 6841, 4131, 3773, 78, 2807, 99, 6462],
 [0.0,
  0.8274081945419312,
  0.9530237913131714,
  0.9788058996200562,
  1.0133435726165771,
  1.0189106464385986,
  1.0396513938903809,
  1.0554074048995972,
  1.0576972961425781,
  1.0597047805786133])

In [26]:
rawdata[['body','gid']].iloc[3560-1]

body    The photo shows doctors who died from COVID-19.
gid                                                3560
Name: 3559, dtype: object

In [22]:
import sqlite3
def get_related_articles(txt):
    vec = tfidf_vectorizer.transform([txt]).toarray()[0]
    hits = tfidf.get_nns_by_vector(vec, 10, search_k=-1, include_distances=True)
    print(hits)
    resp = []
    conn = sqlite3.connect('../../pickles/swarog_data/swarog.sqlite')
    c = conn.cursor()
    for index,_id in enumerate(hits[0]):
        c.execute("select raw.body,raw.label from raw join tfidf on (tfidf.gid=raw.rowid) where tfidf.rowid=?",[_id])
        r=c.fetchall()
        resp.append({
            'text':r[0][0],
            'label':r[0][1],
            'distance':hits[1][index]})
    conn.close()
    return resp
    

In [20]:
rawdata.iloc[95468-1]['body']

"The Hill   During a recent interview, Clooney describes conservative criticism of Democrats associated with film producer Harvey Weinstein as  a little pot and kettle,  noting the prior sexual harassment allegations levied against conservative figures like Roger Ailes and Bill O Reilly. Yeah, that s a little pot and kettle there, unfortunately. But I think that everything gets politicized these days. The reality is that this is a problem deeply ingrained in our society,  Clooney told The Daily Beast when asked about criticisms coming from conservatives in the days since a New York Times report that detailed accusations of sexual harassment against Weinstein. It was something that was talked about a lot on the left with Roger Ailes, Bill O Reilly, and Donald Trump, and it s something that s going to be talked about a lot on the right with Harvey Weinstein,  Clooney added. I think that rather than politicize it, there should be talk on both sides about the really bad behavior by powerfu

In [24]:
rawdata.iloc[69-1]['body']

'Children vaccinated against the flu are at a significantly higher risk of contracting other respiratory infections, including coronaviruses.'

In [23]:
get_related_articles("""
The Hill   During a recent interview, Clooney describes conservative criticism of Democrats associated with film producer Harvey Weinstein as  a little pot and kettle,  noting the prior sexual harassment allegations levied against conservative figures like Roger Ailes and Bill O Reilly. Yeah, that s a little pot and kettle there, unfortunately. But I think that everything gets politicized these days. The reality is that this is a problem deeply ingrained in our society,  Clooney told The Daily Beast when asked about criticisms coming from conservatives in the days since a New York Times report that detailed accusations of sexual harassment against Weinstein. It was something that was talked about a lot on the left with Roger Ailes, Bill O Reilly, and Donald Trump, and it s something that s going to be talked about a lot on the right with Harvey Weinstein,  Clooney added. I think that rather than politicize it, there should be talk on both sides about the really bad behavior by powerful men and the horrible acts they commit. It s pretty crazy to me. Page Six   A former  ER  actress claims that she was sexually and racially harassed daily on set   and George Clooney helped  blacklist  her from Hollywood when she complained.B.S.Clooney helped blacklist me when I spoke up abt harassment on ER.'women who dont play the game lose career'I did https://t.co/ur466NCkII  Ghoul Please   (@vanessathought) October 10, 2017 Clooney helped blacklist me when I spoke up abt harassment on ER. women who don t play the game lose career I did,  tweeted Vanessa Marquez, who played a nurse on the show s first three seasons.#NewPicWhen ur racially&sexually harassed on set daily&go2your producer(John Wells)for help in '95?U r blacklisted from Hollywood 4Ever #ER pic.twitter.com/ptM06gWW7M  Ghoul Please   (@vanessathought) July 7, 2017She accused actor Eriq La Salle and a crew member of being  p sy grabbers  and said racial abuse about her Mexican heritage came from  Anthony, Noah, Julianna,  referring to cast members, Anthony Edwards, Noah Wyle, and Julianna Margulies.Marquez said she complained to executive producer John Wells. Wells was the boss&I 1st reported it to him.His 1st question: Did George do something to u,  she tweeted.
""" )

([86528, 94657, 94668, 86552, 94672, 86256, 94480, 61131, 94675, 86550], [0.0, 0.0, 1.1539806127548218, 1.188228726387024, 1.188228726387024, 1.1944177150726318, 1.1944177150726318, 1.2074379920959473, 1.2116031646728516, 1.2157697677612305])


[{'text': "The Hill   During a recent interview, Clooney describes conservative criticism of Democrats associated with film producer Harvey Weinstein as  a little pot and kettle,  noting the prior sexual harassment allegations levied against conservative figures like Roger Ailes and Bill O Reilly. Yeah, that s a little pot and kettle there, unfortunately. But I think that everything gets politicized these days. The reality is that this is a problem deeply ingrained in our society,  Clooney told The Daily Beast when asked about criticisms coming from conservatives in the days since a New York Times report that detailed accusations of sexual harassment against Weinstein. It was something that was talked about a lot on the left with Roger Ailes, Bill O Reilly, and Donald Trump, and it s something that s going to be talked about a lot on the right with Harvey Weinstein,  Clooney added. I think that rather than politicize it, there should be talk on both sides about the really bad behavior 